!pip install editdistance

In [1]:
%load_ext autoreload
%autoreload 2

import os
import datetime as dt

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import models
import encoders
import decoders
#from training import train, test, checkpoint, main_wandb, main_simple
from dataloaders import load_data, make_loaders, append_SOS

from MADE import MADE, main, main_wandb

import sys
sys.path.insert(0, '/scratch/users/udemir15/ELEC491/bassline_transcription')
from utilities import *

from bassline_transcriber.transcription import NN_output_to_MIDI_file

from plotting import F0_related_stem

from utilities import *

SEED = 27

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

wandb: Currently logged in as: raraz15 (use `wandb login --relogin` to force relogin)


cuda


In [2]:
directories = get_directories('/scratch/users/udemir15/ELEC491/bassline_transcription/data/directories.json')

In [38]:
M = 8

data_params = {'dataset_path': '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/groove', #/single_octave/no_sus', #[28, 51]
               'dataset_name': 'TechHouse_bassline_representations_4521', #4020
               'scale_type': 'min',
               'M': M}

X, titles = load_data(data_params)
#X = append_SOS(X)

K = X.max()+1 # Number of classes, assumes consecutive [0,max] inclusive
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

Number of classes: 2
Sequence Length: 64
Number of data points: 4521


In [39]:
train_params = {'batch_size': 64,
               'N_epochs': 150,
               'lr': 1e-3}

params = {'training': train_params, 'hidden_size': [512]*4, 'data': data_params}

train_loader, validation_loader = make_loaders(X, train_params['batch_size'])

model = MADE((X.shape[1],), K, params['hidden_size'], one_hot_input=True).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=train_params['lr'])

#best_model = main(model, optimizer, device, train_loader, validation_loader, train_params['N_epochs'], True)
best_model = main_wandb(model, optimizer, device, train_loader, validation_loader, params, project='MADE',return_best=True)

  1%|          | 1/150 [00:00<01:37,  1.53it/s]

Epoch: 1, train_loss: 0.666494, validation_loss: 0.651881
[1 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 1 0 1 0 1 1 1 1
 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 0 1 0 1 0 1]


 17%|█▋        | 26/150 [00:07<00:33,  3.74it/s]

Epoch: 26, train_loss: 0.472170, validation_loss: 0.495786
[0 0 1 1 1 0 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 0 0 1 1
 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 0 1 0 1]


 34%|███▍      | 51/150 [00:13<00:24,  4.02it/s]

...

 84%|████████▍ | 126/150 [00:31<00:05,  4.75it/s]

Epoch: 126, train_loss: 0.336358, validation_loss: 0.729926
[0 1 0 0 1 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 1 0 1 1 1 0 1 1 1 0 1 1 0 0 1 0 0
 0 1 0 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1]


100%|██████████| 150/150 [00:36<00:00,  4.14it/s]


train_loss,0.3131
validation_loss,0.85885
_runtime,48
_timestamp,1623330100
_step,155


train_loss,█▇▆▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
validation_loss,▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▅▅▆▆▆▆▇██
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [29]:
samples = best_model.sample(10)
print(samples)

[[ 4  3  0  8  5  0  0  0  0  0  0 12  2  3  5 12  0 11  0  0  5  6  3  0
   0  0  0  0  5  0  0  5 10  0  0  5  5  0  5  0  6  0  0  5  5  0  0  5
   5  0  0  5  7  0  0  7  0  0  0  5  6  0  0  5]
 [ 0  0  0  9  0  0  0  4  0  0  0  1  1  0  9  4  0  0  7  0  0  2  0  4
   0  0 11  0  0  0  8  0  0  0 11  1  0  0  5  0  7  0  0  1  1  1  1  1
   0 11  0  1  0  0  0  0  8  0  0  1  1  0  0  8]
 [ 0  0  0  1  0  2  6  0  6  0  0  4  8  1  0  1 11  0  0  0  0  9  6  2
   3  0 11  1  0 11  0  1  1  2  0  0  5  1  4  2  5  0 12  0  7  1  1  9
   1  0  0 12 11  0  0  1  4  4  1  0  1  0  1  1]
 [10  0  4  9  4  4  3 12  0  0  4  8 11  0  3  1  0  0 10  3  9  1  0  4
   0  4  6 11  6  0  3  0  0 10  0  4  0  3  3  5  1  4  3  9  7  4  1  0
   4  0  2  3  3  3  0  0  3  3  0  0  5  0  6  4]
 [ 8  0  0  0  0  0  0  0  4  0 10  6  3  3  0  0  0  4  9  6 11  4  6  0
   0  0  0 11  6  6  0 11  0  0  0  0  0  0  4  0  9  5  0  4  7  0  4  4
  11  4  4  0  0  6  4  0  4  2  0  4  4  6  0 11]
 [ 0 

In [ ]:
print_beat_matrix(s, 8)

bassline = samples[0]
print_beat_matrix(bassline, M=8)
print_transposed_beat_matrix(bassline, M=8)

In [40]:
samples = best_model.sample(10)
print(samples)

[[0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1
  1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1]
 [0 1 0 0 0 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 1 1 1 0 0 0
  0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 1
  1 0 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1]
 [1 0 0 1 0 1 0 1 0 1 1 1 0 0 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 0 1 1 1 1 1 0
  0 0 1 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1]
 [0 0 1 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 0 1 1
  1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1]
 [0 0 1 0 1 0 0 1 1 0 1 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0
  1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 0 1 0 0 1 1 1]
 [0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 0 0 0 1 0 1 0 1
  1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1]
 [0 1 1 0 0 1 1 0 0 1 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0 1 1 0 1 1 1 1 0 1

In [42]:
samples

array([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
        0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1],
       [1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,

In [41]:
min_note = 36
sustain_code = None
for i, sample in enumerate(samples):
    NN_output_to_MIDI_file(sample, '{}'.format(i), 'midi/made/groove', M,
                           min_note=min_note,
                           sustain_code=sustain_code)

## Bar Levehnsteign

In [ ]:
import editdistance as ed

In [ ]:
def measure_beat_distances(bassline):
    
    _bassline = bassline.reshape(-1, 4)
    
    distances = {}
    for i, beat in enumerate(_bassline):
        for j in range(i+1, 16):
            distances['{}-{}'.format(i,j)] = (ed.eval(beat, _bassline[j,:]))
    
    return distances
            
def measure_bar_distances(bassline):
    _bassline = bassline.reshape(-1, 16)
    
    distances = {}
    for i, beat in enumerate(_bassline):
        for j in range(i+1, 4):

            distances['{}-{}'.format(i,j)] = (ed.eval(beat, _bassline[j,:]))   
            
    return distances

def plot_distances(bar_distances, beat_distances):
    
    fig, ax = plt.subplots(nrows=2, figsize=(20,8), constrained_layout=True)
    ax[0].set_title('Bar Distances')
    ax[0].bar(bar_distances.keys(),bar_distances.values())
    ax[0].tick_params(axis='x', rotation=90)
    ax[1].bar(beat_distances.keys(),beat_distances.values())
    ax[1].tick_params(axis='x', rotation=90)
    ax[1].set_title('Beat Distances')
    plt.show()    

In [ ]:
bassline = bassline.reshape(4,4,4)
bar_distances = measure_bar_distances(bassline)
beat_distances = measure_beat_distances(bassline)

In [ ]:
plot_distances(bar_distances, beat_distances)

## On-Off

In [ ]:
def replace_sustain(arr, cont_token):
    for r in arr:
        for idx, el in enumerate(r[1:]):
            if el == cont_token:
                r[idx + 1] = r[idx]

In [ ]:
M = 8

data_params = {'dataset_path': '/scratch/users/udemir15/ELEC491/bassline_transcription/data/datasets/[28, 51]',
               'dataset_name': 'TechHouse_bassline_representations_4020',
               'scale_type': 'min',
               'M': M}

X, titles = load_data(data_params)
replace_sustain(X, 25)
X[X!=0] = 1

K = X.max()+1 # Number of classes, assumes consecutive [0,max] inclusive
sequence_length = X.shape[1]

print('Number of classes: {}\nSequence Length: {}'.format(K, sequence_length))
print('Number of data points: {}'.format(X.shape[0]))

In [ ]:
project_name = 'groove_MADE'

train_params = {'batch_size': 16,
               'N_epochs': 300,
               'lr': 1e-3}

params = {'training': train_params, 'hidden_size': [32]*2}

train_loader, validation_loader = make_loaders(X, train_params['batch_size'])

model = MADE((X.shape[1],), K, params['hidden_size'], one_hot_input=True).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=train_params['lr'])

#best_model = main(model, optimizer, device, train_loader, validation_loader, train_params['N_epochs'], True)
best_model = main_wandb(model, optimizer, device, train_loader, validation_loader, params, project_name, True)

In [ ]:
N=10

sample = best_model.sample(N)

sample[sample==1] = 15
sample = sample.reshape(N,-1)

put_sustain(sample.reshape(N,-1), 100)

In [ ]:
out_dir = 'midi/groove_made'
os.makedirs(out_dir, exist_ok=True)

for idx, s in enumerate(sample):
    
    NN_output_to_MIDI_file(s, 125, '{}'.format(idx), out_dir, M)
    

In [ ]:
print_beat_matrix(sample, M=8)
print_transposed_beat_matrix(sample, M=8)

In [ ]:
sample

In [ ]:
bassline = sample[0].reshape(4,4,4)
bar_distances = measure_bar_distances(bassline)
beat_distances = measure_beat_distances(bassline)

In [ ]:
plot_distances(bar_distances, beat_distances)

In [ ]:
def put_sustain(arr, cont_token):
    for r in arr:
        for idx in range(len(r))[::-1][:-1]:
            if r[idx] == r[idx - 1]:
                r[idx] = cont_token

In [ ]:
from torch.utils.data import DataLoader
from dataloaders import DataSet

def make_loaders(x_train, x_val, batch_size=32):
    train_set, validation_set = DataSet(x_train), DataSet(x_val)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False, drop_last=True)
    validation_loader = DataLoader(validation_set, batch_size=batch_size, shuffle=True, drop_last=True)
    
    return train_loader, validation_loader

In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(shuffle=True, random_state=42)

In [ ]:
import torch.nn.functional as F

train_indices, val_indices = [], []
train_losses, val_losses = [], []
best_models = []

for train_index, val_index in kfold.split(X, X):  
    train_indices.append(train_index)
    val_indices.append(val_index)

    x_train_fold = X[train_index]
    x_val_fold = X[val_index]
    train_params = {'batch_size': 32,
                   'N_epochs': 50,
                   'lr': 1e-3}

    train_loader, validation_loader = make_loaders(x_train_fold, x_val_fold, train_params['batch_size'])
    model = MADE((X.shape[1], ), K, hidden_size=[512]*2, one_hot_input=True).cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=train_params['lr'])

    best_model = main(model, optimizer, device, train_loader, validation_loader, None, train_params['N_epochs'], True)
    best_models.append(best_model)
    
    cat_cross = nn.CrossEntropyLoss()
    train_fold_tensor = torch.tensor(x_train_fold).to(device)
    val_fold_tensor = torch.tensor(x_val_fold).to(device)
    train_losses.append(F.cross_entropy(best_model(train_fold_tensor), train_fold_tensor, reduction='none'))
    val_losses.append(F.cross_entropy(best_model(val_fold_tensor), val_fold_tensor, reduction='none'))

In [ ]:
losses_arr = []

for train_index, train_loss, val_index, val_loss in zip(train_indices, train_losses, val_indices, val_losses):
    losses_arr.append(val_loss.mean(dim=1).cpu().detach().numpy())

losses_arr = np.concatenate(losses_arr)

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

def hist(x, binwidth=1, title='', ax=None, return_fig=False):
    #fig = plt.figure(figsize=(30, 12))
    lspace = np.arange(np.min(x), np.max(x) + binwidth, binwidth)
    if ax is None:
        plt.hist(x, bins=lspace, linewidth=1, histtype='bar', ec='black', density=True)
        plt.title(title, fontdict={'fontsize':30})
        plt.xlabel('Random Variable', fontdict={'fontsize':20})
        plt.ylabel('Density', fontdict={'fontsize':20})
        plt.xticks(np.arange(np.floor(np.min(x)), np.ceil(np.max(x)), 1), fontsize=20, rotation=0)
        plt.yticks(fontsize=20)
    else:
        ax.hist(x, bins=lspace, linewidth=1, histtype='bar', ec='black', density=True)
        kde = gaussian_kde(x)
        ls = np.linspace(x.min(), x.max(), 1000)
        pdf = kde(ls)
        ax.plot(ls, pdf)
    return fig if return_fig else None

hist(losses_arr, binwidth=0.1)

In [ ]:
losses_argsorted = losses_arr.argsort()[::-1]

losses_sorted_indices = np.concatenate(val_indices)[losses_argsorted]

In [ ]:
songs = np.column_stack((np.array(titles)[losses_sorted_indices], losses_arr[losses_argsorted]))

In [ ]:
songs

In [ ]:
leftover_songs = np.array(titles)[losses_sorted_indices[:-400]]

In [ ]:
leftover_songs = np.insert(leftover_songs[2:], 0, leftover_songs[0])

In [ ]:
leftover_songs = leftover_songs.tolist()

In [ ]:
with open('leftover_tracks.txt', 'w') as f:
    f.writelines('\n'.join(leftover_songs))

In [ ]:
with open('leftover_tracks.txt', 'r') as f:
    print(f.readlines())

In [ ]:
removed_tracks = np.array(titles)[losses_sorted_indices[-400:]]
removed_tracks = np.insert(removed_tracks, 0, np.array(titles)[losses_sorted_indices[1]])

In [ ]:
removed_tracks = removed_tracks.tolist()

In [ ]:
with open('removed_tracks.txt', 'w') as f:
    f.writelines('\n'.join(removed_tracks))

In [ ]:
with open('removed_tracks.txt', 'r') as f:
    print(f.readlines())

In [ ]:
idx = -1

title = songs[idx,0]

inspect_audio_outputs(title, directories)

In [ ]:
song_representation = X[losses_sorted_indices[idx]]

print_beat_matrix(song_representation, M=8)
print_transposed_beat_matrix(song_representation, M=8)

In [ ]:
from bassline_transcriber.transcription import unpack_repetitions

f0_estimate = load_F0_estimate(title, directories)
pitch_track = load_pitch_track(title, directories)
quantized_pitch_track = load_quantized_pitch_track(title, directories)
bassline_midi_number = unpack_repetitions(load_symbolic_representation(title, directories, M=8))
F0_related_stem(title, f0_estimate, pitch_track, quantized_pitch_track, bassline_midi_number, M=8)

In [ ]:
indices.append(idx)

In [ ]:
indices

In [ ]:
plt.bar(np.array(titles)[losses_sorted_indices], height=losses_arr[losses_argsorted])

In [ ]:
plt.hist(losses_arr[losses_argsorted], np.array(titles)[losses_sorted_indices])

In [ ]:
bassline = model.sample(1)

In [ ]:
print_beat_matrix(bassline, M)

In [ ]:
bassline

In [ ]:
bassline=bassline.reshape(64)

In [ ]:
NN_output_to_MIDI_file(bassline, 125, 'made4', '.', 
                            8, M, middle_c='C3', tpb=960*16,
                            min_note=28, silence_code=0, sustain_code=100, velocity=120)

In [ ]:
X.shape